# Periodic Boundaries

<img src="https://upload.wikimedia.org/wikipedia/commons/e/eb/Periodic_Boundary_Conditions_in_2D.png" width="500"/>

When conducting molecular simulations we can only afford to include so many particles.  A common way to minimize the effects of a finite system size is to include **periodic boundaries**.  

This essentially assumes that the simulation cell is surrounded in all directions by other identical simulation cells.

The implementation of periodic boundaries seems simple enough:  
- if you exit through the left side, you enter through the right
- if you exit through the top, you enter through the bottom
- etc.

**But periodic boundaries can be tricky and are probably the cause of most errors and bugs in molecular dynamics simulations.**

Here are a couple of useful functions we will use to handle periodic boundaries in class:

___
## "Wrapping" coordinates

To apply those rules above and make sure particles do not leave the simulation box, you need to periodically "wrap" the coordinates back into the box.

Below is trajectory that leaves a simulation box, defined in two dimensions as $x \in [-0.5,0.5]$, $y \in [-0.5,0.5]$:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# an imaginary trajectory
traj = np.zeros((20,2))
traj[:,0] = np.linspace(0,0.7,20)
traj[:,1] = 0.2*np.sin(8*traj[:,0])

# used for visualizing the box
box_x = [-0.5,0.5,0.5,-0.5,-0.5]
box_y = [-0.5,-0.5,0.5,0.5,-0.5]

plt.figure(figsize=[5,5])
plt.plot(traj[:,0],traj[:,1],'bo')
plt.plot(box_x,box_y)
plt.xlim([-1,1])
plt.ylim([-1,1])


Here is a function that applies boundary conditions to a given trajectory frame and returns the "wrapped" positions back inside the box:

In [ ]:
def apply_pbc(positions, box_size):
    # positions and box size are both arrays with
    # a dimensionality equal to the number of spatial dimensions
    # i.e. if we are in 2D space, each array has two elements
    
    half_box_size = box_size * 0.5
    pbc_wrap_positions = np.zeros_like(positions)

    old_positions = positions

    # this while loop allows adjustments to be made until 
    # the positions of the particles stops changing
    while np.any(pbc_wrap_positions != old_positions):
        #
        # Note on np.where syntax:
        # np.where(CONDITION, value_if_true, value_if_false)
        #
        pbc_wrap_positions = np.where(positions > half_box_size,
                                      positions - box_size,
                                      positions)

        pbc_wrap_positions = np.where(positions <= -half_box_size,
                                      positions + box_size,
                                      pbc_wrap_positions)
        old_positions = pbc_wrap_positions
    return pbc_wrap_positions

Let's try this out by applying it to all of the frames in our trajectory:

In [ ]:
new_frames = []
box_size = np.array([1,1])
for frame in traj:
    new_frames.append(apply_pbc(frame,box_size))

new_traj = np.array(new_frames)
plt.figure(figsize=[5,5])
plt.plot(new_traj[:,0],new_traj[:,1],'bo')
plt.plot(box_x,box_y)
plt.xlim([-1,1])
plt.ylim([-1,1])

**Great!  Note that we can use the same function with 3D data as well:**

In [ ]:
unwrapped_trajectory = np.random.random(size=(100,10,3))*10  # nframes, nparticles, ndim
box_size = np.array([5,5,5])

print("Max x coordinate:",unwrapped_trajectory[:,:,0].max())
print("Max y coordinate:",unwrapped_trajectory[:,:,1].max())
print("Max z coordinate:",unwrapped_trajectory[:,:,2].max())

In [ ]:
wrapped_trajectory = apply_pbc(unwrapped_trajectory,box_size)

print("Max x coordinate (after wrapping):",wrapped_trajectory[:,:,0].max())
print("Max y coordinate (after wrapping):",wrapped_trajectory[:,:,1].max())
print("Max z coordinate (after wrapping):",wrapped_trajectory[:,:,2].max())

___
## Displacement vectors through periodic boundaries

Wrapping is important, but it is also important that inter-particle forces are computed properly through periodic boundaries.  In other words, **even though the positions of two particles are far apart, their "images" in adjacent cells could be close together.**  If these interactions aren't taken into account, particles could experience discontinuities in their forces as they are wrapped through the periodic boundary.  **For instance, a particle would know that it is sitting almost on top of another particle until it crosses the boundary, at which point it suddenly experiences an ultra-high repulsive force, and then your system explodes!**

To avoid this catastrophic scenario, we will make sure to compute forces using the closest possible displacement vector.

Consider two particles $i$ and $j$:

![periodicity schematic](https://github.com/ADicksonLab/ml4md-jb/blob/main/figures/periodic-01.png?raw=true)

The difference in their $x$ coordinates can be calculated three ways:

-  directly ($x_j - x_i$)
-  using the "right hand image" ($x_j - (x_i + L_x)$)
-  using the "left hand image" ($x_j - (x_i - L_x)$)

where $L_x$ is the length of the box in the $x$ direction.

**Clearly if we want $i$ and $j$ to "feel" each other across the periodic boundary, we need to use the difference with the smallest magnitude when we are calculating the forces.**

The following function determines the shortest displacement vector between two atoms, **even considering images that differ by more than one box length**.

In [ ]:
def shortest_distance(posA, posB, box_size):

    r = posA - posB   # has one element for each spatial dimension

    # loop over spatial dimensions
    for i in range(len(posA)):
        
        # if difference is greater than half the box length, adjust
        if r[i] < -0.5*box_size[i]:
            
            # determine number of adjustments needed
            n = int((r[i] + 0.5*box_size[i])/box_size[i]) + 1  

            # make them
            r[i] += n*box_size[i]
        elif r[i] > 0.5*box_size[i]: # same thing in the negative direction
            n = int((r[i] - 0.5*box_size[i])/box_size[i]) + 1
            r[i] -= n*box_size[i]

    return r

**Now let's test this out:**

In [ ]:
positions = np.array([[0.01,0.52,0.55],
                      [0.66,0.31,0.77],
                      [0.90,0.51,0.2],
                      [0.90,0.45,0.50],
                      [0.2,0.77,0.9]])   # 5 particles in 3D space
box_size = [1,1,1]

for i in range(N-1):
    for j in range(i+1,N):
        direct_r = np.sqrt(np.sum(np.square(positions[i] - positions[j])))
        closest_r = np.sqrt(np.sum(np.square(shortest_distance(positions[i],positions[j],box_size))))
        
        if closest_r < direct_r:
            print(f"Particle {i}: ({positions[i][0]:.2},{positions[i][1]:.2},{positions[i][2]:.2}) and {j}: ({positions[j][0]:.2},{positions[j][1]:.2},{positions[j][2]:.2}) are closer through a boundary: {closest_r:.2} < {direct_r:.2}")
        else:
            print(f"Particle {i}: ({positions[i][0]:.2},{positions[i][1]:.2},{positions[i][2]:.2}) and {j}: ({positions[j][0]:.2},{positions[j][1]:.2},{positions[j][2]:.2}) are closest in direct space.")

**Task: look through this output.  Does this make sense?  Can you think of another way to test this function?**